## Load modules

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.autograd import Variable
import time
import os
from torchvision import transforms
import shutil
from dataloader import *
from models.vgg import VGG11

## Save dog/cat data

In [47]:
transform = transforms.Compose(
    [transforms.ToTensor()])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

dog_idx = (np.asarray(testset.targets) == 5)
testset.targets = np.asarray(testset.targets)[dog_idx]
testset.data = np.asarray(testset.data)[dog_idx]

Files already downloaded and verified


In [49]:
torch.save(testset, './data/dog.pt')

In [50]:
transform = transforms.Compose(
    [transforms.ToTensor()])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

cat_idx = (np.asarray(testset.targets) == 3)
testset.targets = np.asarray(testset.targets)[cat_idx]
testset.data = np.asarray(testset.data)[cat_idx]

Files already downloaded and verified


In [51]:
torch.save(testset, './data/cat.pt')

## Save adversarial data which final pred is cat

In [57]:

def adv_attack(image, epsilon, data_grad, attack_method):
    assert attack_method in ['fgsm', 'stepll']

    # Collect the element-wise sign of the data gradient
    sign_data_grad = torch.sign(data_grad)

    # Create the perturbed image by adjusting each pixel of the input image
    if attack_method == 'fgsm':
        perturbed_image = image + epsilon * sign_data_grad
    else:
        perturbed_image = image - epsilon * sign_data_grad

    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image


def fgsm_attack(model, device, dataloader, criterion, attack_method, epsilon):
    assert attack_method in ['fgsm', 'stepll']

    # Accuracy counter
    correct = 0
    total = 0
    adv_examples = []
    ct_save = 0
    adv_cat = torch.tensor([])

    # Loop over all examples in test set
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output, _ = model(data)
        init_pred = output.max(1, keepdim=True)[1]  # get the index of the max log-probability

        if init_pred.item() != target.item():  # initially was incorrect --> no need to generate adversary
            continue

        if attack_method == 'fgsm':
            loss = criterion(output, target)  # loss for ground-truth class
        else:
            ll = output.min(1, keepdim=True)[1][0]
            loss = criterion(output, ll)  # Loss for least-likely class

        # Back propogation
        model.zero_grad()
        loss.backward()

        # Collect data_grad
        data_grad = data.grad.data

        # Call Attack
        perturbed_data = adv_attack(data, epsilon, data_grad, attack_method)

        # Re-classify the perturbed image
        model.eval()
        with torch.no_grad():
            output, _ = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1]
        if final_pred.item() == target.item():
            correct += 1  # still correct

        else:  # attack is successful
            if final_pred.item() == 3:
                adv_cat = torch.cat([adv_cat, perturbed_data.detach().cpu()], dim=0)

        # Special case for saving 0 epsilon examples
        adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
        adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))
        total += 1
    
    torch.save(adv_cat, './data/adv_cat.pt')
    # Calculate final accuracy for this epsilon
    final_acc = correct / float(len(dataloader))


In [53]:
'''define model'''
print("CUDA Available: ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the network
model = VGG11()
model.to(device)

'''load saved model'''
model.load_state_dict(torch.load('./checkpoints/model.th')['state_dict'])
model.eval()

CUDA Available:  True


VGG11(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv4): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, 

In [58]:
criterion = nn.CrossEntropyLoss()
fgsm_attack(model, device, testloader, criterion, 'fgsm', epsilon=0.05)

Epsilon: 0.05	Test Accuracy = 193 / 7867 = 0.0193
